In [1]:
!pip install -q bitsandbytes accelerate transformers datasets evaluate scikit-learn gradio


In [3]:
!pip install -U bitsandbytes accelerate


In [20]:
print(train_ds.column_names)


['image', 'label']


In [1]:
import torch
from transformers import ViTForImageClassification, TrainingArguments, Trainer, ViTImageProcessor
from datasets import load_dataset
from sklearn.metrics import accuracy_score
import numpy as np

# --- Load Food-41 dataset ---
dataset = load_dataset("imagefolder", data_dir="/kaggle/input/food41/images")
dataset = dataset["train"].train_test_split(test_size=0.1)
train_ds, val_ds = dataset["train"], dataset["test"]



2025-11-02 16:54:28.611970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762102468.803050      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762102468.862927      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Resolving data files:   0%|          | 0/101000 [00:00<?, ?it/s]

Computing checksums:  12%|#1        | 12013/101000 [00:05<00:37, 2402.54it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [24]:
# Check column names
print(train_ds.column_names)
# Example output: ['image', 'label']


['image', 'label']


In [2]:
from transformers import ViTImageProcessor, ViTForImageClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import numpy as np
from sklearn.metrics import accuracy_score
from PIL import Image

# ------------------------------
# 1️⃣ Processor
# ------------------------------
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

# ------------------------------
# 2️⃣ Transform function
# ------------------------------
def transform(batch):
    """
    Transform batch images safely for ViT.
    Ensures images are RGB and compatible with processor.
    """
    if "image" in batch:
        processed_images = []
        for img in batch["image"]:
            # Convert to PIL if it's not
            if not isinstance(img, Image.Image):
                img = Image.fromarray(img)
            # Ensure RGB
            if img.mode != "RGB":
                img = img.convert("RGB")
            processed_images.append(img)
        
        # Process images with ViT processor
        processed = processor(images=processed_images, return_tensors="pt")
        processed["labels"] = batch["label"]  # Keep labels
        # Convert tensors to lists for HuggingFace Dataset compatibility
        return {k: v.tolist() for k, v in processed.items()}
    else:
        return batch

# ------------------------------
# 3️⃣ Map datasets
# ------------------------------
# Replace with your actual datasets
train_ds = train_ds.map(transform, batched=True, remove_columns=["image", "label"])
val_ds   = val_ds.map(transform, batched=True, remove_columns=["image", "label"])

# ------------------------------
# 4️⃣ Load model
# ------------------------------
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=41,             # Change for your dataset
    ignore_mismatched_sizes=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

# ------------------------------
# 5️⃣ Freeze ViT backbone
# ------------------------------
for param in model.vit.parameters():
    param.requires_grad = False

print("✅ ViT backbone frozen. Only classifier head will be trained.")
print("Trainable parameters:", sum(p.requires_grad for p in model.parameters()))

# ------------------------------
# 6️⃣ Metrics
# ------------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, preds)}

# ------------------------------
# 7️⃣ Training arguments
# ------------------------------
training_args = TrainingArguments(
    output_dir="./vit-fp16-food41",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    fp16=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    logging_steps=50,
    report_to="none",
)

# ------------------------------
# 8️⃣ Trainer
# ------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
)

# ------------------------------
# 9️⃣ Train
# ------------------------------
trainer.train()

# ------------------------------
# 🔟 Evaluate
# ------------------------------
results = trainer.evaluate()
print("✅ Final Evaluation Results:", results)


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Map:   0%|          | 0/90900 [00:00<?, ? examples/s]

AttributeError: 'list' object has no attribute 'tolist'

In [ ]:
metrics = trainer.evaluate()
print(metrics)


In [30]:
def transform(batch):
    print(type(batch))
    print(batch.keys() if hasattr(batch, "keys") else batch[:5])
    return batch  # temporarily just return the batch


In [31]:
train_ds.select(range(5)).map(transform, batched=True)


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

<class 'transformers.image_processing_base.BatchFeature'>
KeysView({'pixel_values': tensor([[[[ 0.1529,  0.1294,  0.1373,  ...,  0.3961,  0.4353,  0.4196],
          [ 0.1373,  0.1373,  0.1451,  ...,  0.4510,  0.4980,  0.4510],
          [ 0.1529,  0.1529,  0.1608,  ...,  0.4353,  0.4118,  0.4118],
          ...,
          [-0.3725, -0.3804, -0.3882,  ...,  0.4745,  0.4824,  0.4902],
          [-0.2784, -0.3333, -0.3176,  ...,  0.4824,  0.4824,  0.4980],
          [ 0.0039, -0.1294, -0.1294,  ...,  0.4980,  0.4824,  0.4588]],

         [[-0.3020, -0.3176, -0.3176,  ...,  0.0118,  0.0510,  0.0353],
          [-0.3412, -0.3333, -0.3333,  ...,  0.0588,  0.1137,  0.0667],
          [-0.3333, -0.3333, -0.3255,  ...,  0.0353,  0.0039,  0.0118],
          ...,
          [-0.2549, -0.2471, -0.2549,  ..., -0.0510, -0.0431, -0.0275],
          [-0.2706, -0.3176, -0.2941,  ..., -0.0431, -0.0431, -0.0196],
          [-0.0902, -0.2078, -0.2000,  ..., -0.0275, -0.0353, -0.0510]],

         [[-0.8431

Dataset({
    features: ['image', 'label', 'pixel_values', 'labels'],
    num_rows: 5
})